# CMF Rateslib Project

### Conventions
 - 1 year == 1.0
 - 1 month == 1/12
 - 1 day == 1/252
 - 0.02 == 2%

In [ ]:
cmf_rateslib_root = '../'
import sys
if cmf_rateslib_root not in sys.path:
    sys.path.append(cmf_rateslib_root)

In [ ]:
import numpy as np

# Curves

In [ ]:
from cmf_rateslib.curves.base_curve import BaseZeroCurve
from cmf_rateslib.curves.zero_curve import ZeroCurve

In [ ]:
%pylab
%matplotlib inline

### Interpolation

In [ ]:
curve = ZeroCurve([0.1, 0.2, 0.3, 0.5, 0.7, 1, 2, 3, 5, 10], [0.0751, 0.0763, 0.0773, 0.079, 0.0804, 0.0819, 0.0838, 0.0838, 0.0825, 0.0808])
curve.lin_interpolate('zero_rates')
curve.plot()

In [ ]:
curve = ZeroCurve([0.1, 0.2, 0.3, 0.5, 0.7, 1, 2, 3, 5, 10], [0.0751, 0.0763, 0.0773, 0.079, 0.0804, 0.0819, 0.0838, 0.0838, 0.0825, 0.0808])
curve.lin_interpolate('log_discount_factors')
curve.plot()

In [ ]:
curve = ZeroCurve([0.1, 0.2, 0.3, 0.5, 0.7, 1, 2, 3, 5, 10], [0.0751, 0.0763, 0.0773, 0.079, 0.0804, 0.0819, 0.0838, 0.0838, 0.0825, 0.0808])
curve.lin_interpolate('forward_rates')
curve.plot()

### Rolldown

In [ ]:
curve = ZeroCurve([0.1, 0.2, 0.3, 0.5, 0.7, 1, 2, 3, 5, 10], [0.0751, 0.0763, 0.0773, 0.079, 0.0804, 0.0819, 0.0838, 0.0838, 0.0825, 0.0808])
curve.roll(0.25)
curve.plot()